<h1>Correlation coefficients load</h1>

In [81]:
import pickle
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

correlation_coefficient = pickle.load(open("correlation_coefficient_13.p", "rb"))

cols = ["monster_1_id", "monster_2_id", "Correlation"]
correlation_coefficient = pd.DataFrame(correlation_coefficient, columns=cols).sort('Correlation', ascending=False)
correlation_coefficient.head()

,monster_1_id,monster_2_id,Correlation
11,1031,1044,0.875952
228,1044,1031,0.875952
31,1039,1052,0.779704
248,1052,1039,0.779704
142,1032,1051,0.758424


<h1>Test data load and preprocess</h1>

In [82]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')


test_data = pd.read_csv("../datasets/event_users_test.csv", dtype={'user_id': np.int64})

scaler = preprocessing.MinMaxScaler()
count_scaled = scaler.fit_transform(test_data['count_monster'])
test_data['rating'] = pd.DataFrame(count_scaled)

print "There are %d rows in this dataframe" %len(test_data)

test_data.head()

There are 944605 rows in this dataframe


,user_id,monster_id,count_monster,rating
0,5000065336,2069,2,0.001081
1,5000065336,2073,1,0.000000
2,5000065336,1044,9,0.008649
3,5000065336,2071,5,0.004324
4,5000065336,1046,4,0.003243


In [83]:
test_users_ids = test_data['user_id']
test_users_ids = test_users_ids.unique()

<h1>Top monsters list</h1>

In [84]:
# Top 20 most used monsters 
top_monsters = pd.value_counts(test_data.monster_id)
top_monsters.head()

1031    23100
1039    22985
1045    22962
2019    22898
2017    22843
Name: monster_id, dtype: int64

<h1>Submit file generation</h1>

In [85]:
suggested_monsters_ids = []

test_users_ids_subset = test_users_ids[:100]

# count = 1
for user_id in test_users_ids:
    # print count
    # count = count + 1
    # print "Bucle #1"
    user_id_rows = test_data.loc[test_data['user_id'] == user_id]
    user_id_rows = user_id_rows.sort(['rating'], ascending=[0])
    recommendedMonsterFound = False
    for tuple in user_id_rows.itertuples(): # we loop in rating descending until we found valid monster_id recommendation
        if recommendedMonsterFound == True:
            break
        # print "\tBucle #2"
        monster_id = tuple[2]
        monster_id_correlations = correlation_coefficient.loc[correlation_coefficient['monster_1_id'] == monster_id]
        for correlation_tuple in monster_id_correlations.itertuples():
            # print "\t\tBucle #3"
            suggested_monster_id = correlation_tuple[2]
            hasAlreadySuggestedMonster =  any(user_id_rows.monster_id == suggested_monster_id)
            if hasAlreadySuggestedMonster == False:
                suggested_monsters_ids.append(suggested_monster_id)
                recommendedMonsterFound = True
                break
    if recommendedMonsterFound == False:
        for top_monster in top_monsters.iteritems():
            top_monster_id = top_monster[0]
            hasAlreadySuggestedMonster =  any(user_id_rows.monster_id == suggested_monster_id)
            if hasAlreadySuggestedMonster == False:
                suggested_monsters_ids.append(top_monster_id)
                recommendedMonsterFound = True
                break
    if recommendedMonsterFound == False:
        suggested_monsters_ids.append(2284)
        
print "Done!!!!"

Done!!!!


In [86]:
print len(suggested_monsters_ids)

30000


In [87]:
np.savetxt('jcsastre_submit_13.csv', suggested_monsters_ids, fmt='%i')